# Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

План выполнения проекта:
1. Загрузить и изучите данные.
2. Умножим признаки на обратимую матрицу. Ответим на вопрос, изменится ли качество линейной регрессии? Обоснуем решение.
   *  Изменится. Приведите примеры матриц.
   *  Не изменится. Укажите, как связаны параметры линейной регрессии в исходной задаче и в преобразованной.
3. Предложим алгоритм преобразования данных для решения задачи. Обоснуем, почему качество линейной регрессии не поменяется.
4. Запрограммируем этот алгоритм, применив матричные операции. Проверим, что качество линейной регрессии из sklearn не отличается до и после преобразования. Примените метрику R2.



## Загрузка данных

In [1]:
#импортируем библиотеки
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

***Открываем документ и выводим информацию о файле:***

In [2]:
insurance_data = pd.read_csv('/datasets/insurance.csv')
insurance_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


In [3]:
insurance_data.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


***Выводим уникальные значения в столбцах с помощью функции:***

In [4]:
def show_info(data_frame):
    for column in data_frame.columns:
        print('Уникальные значения столбца:', column)
        print(data_frame[column].unique())
    print('Количество пропусков в каждом столбце:')    
    print(data_frame.isna().mean())
    
print(show_info(insurance_data))

Уникальные значения столбца: Пол
[1 0]
Уникальные значения столбца: Возраст
[41. 46. 29. 21. 28. 43. 39. 25. 36. 32. 38. 23. 40. 34. 26. 42. 27. 33.
 47. 30. 19. 31. 22. 20. 24. 18. 37. 48. 45. 44. 52. 49. 35. 56. 65. 55.
 57. 54. 50. 53. 51. 58. 59. 60. 61. 62.]
Уникальные значения столбца: Зарплата
[49600. 38000. 21000. 41700. 26100. 41000. 39700. 38600. 49700. 51700.
 36600. 29300. 39500. 55000. 43700. 23300. 48900. 33200. 36900. 43500.
 36100. 26600. 48700. 40400. 38400. 34600. 34800. 36800. 42200. 46300.
 30300. 51000. 28100. 64800. 30400. 45300. 38300. 49500. 19400. 40200.
 31700. 69200. 33100. 31600. 34500. 38700. 39600. 42400. 34900. 30500.
 24200. 49900. 14300. 47000. 44800. 43800. 42700. 35400. 57200. 29600.
 37400. 48100. 33700. 61800. 39400. 15600. 52600. 37600. 52500. 32700.
 51600. 60900. 41800. 47400. 26500. 45900. 35700. 34300. 26700. 25700.
 33300. 31100. 31500. 42100. 37300. 42500. 27300. 46800. 33500. 44300.
 41600. 53900. 40100. 44600. 45000. 32000. 38200. 33000. 38

Исходя из полученой информации о данных в файле, необходимо изменить тип данных в столбцах "Возраст" и "Зарплата" на целочисленный, т к дроби отсутствуют. Так же уменьшим обращение к памяти для остальных столбцов путем перевода  их значений в целочисленный формат.

Выведем обновленную информацию о файле.

In [5]:
insurance_data['Пол'] = pd.to_numeric(insurance_data['Пол'], downcast='integer')
insurance_data['Возраст'] = pd.to_numeric(insurance_data['Возраст'], downcast='integer')
insurance_data['Зарплата'] = pd.to_numeric(insurance_data['Зарплата'], downcast='integer')
insurance_data['Члены семьи'] = pd.to_numeric(insurance_data['Члены семьи'], downcast='integer')
insurance_data['Страховые выплаты'] = pd.to_numeric(insurance_data['Страховые выплаты'], downcast='integer')
insurance_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int8   
 1   Возраст            5000 non-null   int8   
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int8   
 4   Страховые выплаты  5000 non-null   int8   
dtypes: float64(1), int8(4)
memory usage: 58.7 KB


In [6]:
X = insurance_data.drop("Страховые выплаты", axis=1)
y = insurance_data["Страховые выплаты"]

X.shape, y.shape

((5000, 4), (5000,))

***Вывод:*** Выполнена загрузка файла.Изучена информация о содержащихся данных. Выполнен перевод данных в целочисленный тип. Аномалий и пропусков не обнаружено.

## Умножение матриц

***Из условия задания известно:***
<br>**Признаки:** пол, возраст и зарплата застрахованного, количество членов его семьи.
<br>**Целевой признак:** количество страховых выплат клиенту за последние 5 лет.

Далле в процессе выполнения задания будем использовать линейную регрессию, т к  регресионная модель, в отличие от классификацтоннной, которая предполагает заранее ограниченное и определенное количество значений целевого признака, может предсказать в случае необходимости и изменяющееся количество значений целевого признака(нам дана информациия за последние 5 лет, но на практике возможен и другой набор значений).

Далее сформируем выборки из данных и проверим работу линейной регрессии из scikit-learn, сравнив ее результат с результатом вычисления по формуле:

In [7]:
features = insurance_data.drop('Страховые выплаты',axis=1)
target = insurance_data['Страховые выплаты']

In [8]:
X = np.concatenate((np.ones((features.shape[0], 1)), features), axis=1)
y = target
w = np.linalg.inv(X.T @ X) @ X.T @ y
display(w[1:])
model = LinearRegression()
model.fit(features, target)
model.coef_

array([ 7.92580543e-03,  3.57083050e-02, -1.70080492e-07, -1.35676623e-02])

array([ 7.92580543e-03,  3.57083050e-02, -1.70080492e-07, -1.35676623e-02])

Коэфициенты регрессии полностью совпадают, что и требовалось доказать.  

***Переходим к теоретической части***

В этом задании вы можете записывать формулы в *Jupyter Notebook.*

Чтобы записать формулу внутри текста, окружите её символами доллара \\$; если снаружи —  двойными символами \\$\\$. Эти формулы записываются на языке вёрстки *LaTeX.* 

Для примера мы записали формулы линейной регрессии. Можете их скопировать и отредактировать, чтобы решить задачу.

Работать в *LaTeX* необязательно.

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

Как получили эту формулу:
- Транспонированная матрица признаков умножается на себя;
- Вычисляется обратная к результату матрица;
- Обратная умножается на транспонированную матрицу признаков;
- Результат умножается на вектор значений целевого признака.


В данной части работы мы должны показать, что умножение на случайную матрицу  не ухудшает метрику R2. С целью защиты информации на этапе поиска коэффициентов регрессии и предсказаний регрессии будем умножать матрицу признаков на обратимую матрицу , которая будет генерироваться случайным образом. Важно, чтобы значение метрики R2 существенно не изменялось, т к существенное изменение будет говорить о том, что произошло искажение и результат не отражает особенности исходных данных.

*Features* и *target* уже определены выше. 

Создадим класс  *my_L_regression* с двумя методами *fit* и *predict*.
Первый находит веса w, второй делает предсказания:

In [9]:
class my_L_regression:
    def fit(self, train_features, train_target):
        X = np.concatenate((np.ones((train_features.shape[0], 1)), train_features), axis=1)
        y = train_target
        w = np.linalg.inv((X.T @ X)).dot(X.T).dot(y)
        self.w = w[1:]
        self.w0 = w[0]
        print(w)
        
    def predict(self, test_features):
        return test_features.dot(self.w) + self.w0

Создадим и обучим модель. Найдем её предсказания на обучающей выборке и сохраним их в переменной predictions. Выведем на экран веса и значение метрики R2.

In [10]:
model =  my_L_regression()
model.fit(features, target)
predictions = model.predict(features)
r2_first = r2_score(target, predictions)
r2_first

[-9.38235504e-01  7.92580543e-03  3.57083050e-02 -1.70080492e-07
 -1.35676623e-02]


0.42494550286668

Далее генерируем случайную матрицу размерностью соответствующую *features*.

In [11]:
random_matrix = np.random.normal(0, 1, (features.shape[1], features.shape[1]))
random_matrix

array([[-0.2040529 , -0.12074638, -0.59945328, -0.91768089],
       [-0.15435865, -1.57268951, -1.24359459,  1.4005207 ],
       [ 0.28980784,  0.6915815 , -0.71832567,  0.24012839],
       [ 1.19124545,  0.00248606,  0.25853541,  0.40859624]])

**Пооверим полученную матрицу на обратимость:** 

*Способ 1*
Умножим исходную матрицу на обратную ей, чтобы получить единичную матрицу:

In [12]:
random_matrix @ np.linalg.inv(random_matrix)

array([[ 1.00000000e+00,  1.01263756e-17, -5.40643937e-17,
        -2.51466728e-17],
       [ 7.60051690e-17,  1.00000000e+00, -2.02134955e-16,
        -8.97120728e-17],
       [-4.57747541e-17, -4.30642622e-17,  1.00000000e+00,
         2.05906420e-17],
       [-2.67657841e-17,  3.57902672e-18,  4.90353802e-18,
         1.00000000e+00]])

*Способ 2*
<br>Для квадратной матрицы  A, обратной является такая матрица A −1, для которой выполняется условие: 
$AA_1=A_1A=E$, где Е - единичная матрица.
<br>Проверим, выполняется ли это условие для сгенерированной матрицы:

In [13]:
if np.allclose(np.dot(random_matrix, np.linalg.inv(random_matrix)), np.eye(random_matrix.shape[0])) & np.allclose(
    np.dot(np.linalg.inv(random_matrix), random_matrix), np.eye(random_matrix.shape[0])) == True:
    print('Условие выполняется')
else:    
    print('Условие не выполняется')


Условие выполняется


**Вывод:** сгененрированная матрица является обратимой.

Умножаем данные из *features* на сгенерированную матрицу:

In [14]:
encoded_features = features.dot(random_matrix)
encoded_features

,0,1,2,3
0,14369.127535,34237.843687,-35680.281658,11967.280656
1,11006.788808,26207.755629,-27353.322376,9189.711558
2,6081.488317,14477.603427,-15120.903368,5083.311395
3,12084.128041,28805.926889,-29979.778964,10043.582199
4,7559.458625,18006.121001,-18783.720157,6305.648008
...,...,...,...,...
4995,10344.200468,24645.429084,-25678.530091,8612.615473
4996,15181.874061,36185.401450,-37682.288926,12630.754198
4997,9823.781219,23413.163907,-24375.595123,8169.180197
4998,9476.690282,22580.002472,-23516.432423,7883.318080


Обучим модель на полученных признаках и вычислим  R-метрику:

In [15]:
model = my_L_regression()
model.fit(encoded_features, target)
predictions = model.predict(encoded_features)
r2_second = r2_score(target, predictions)
r2_second 

[-0.9382355  -0.00966409 -0.00961796 -0.01223309  0.0027686 ]


0.424945502866681

In [16]:
if round(r2_first, 3) == round(r2_second, 3):
    print('R-метрики примерно равны')
else:
    print('R-метрики не равны')

R-метрики примерно равны


**Вопрос:** Как связаны параметры линейной регрессии в исходной задаче и в преобразованной?

Предсказания высчитываются по формуле: $a = Xw$, где 𝑋  — исходная матрица признаков. В процессе работы исходная матрица была умножена на рандомную матрицу, которую можно обозначить как **R**. 
<br>Подставим новый множитель в формулу: $a=XR\omega$
<br>Получаем такую формулу обучения: <font color='lightgray'>$\omega_{new}=((XR)^{T}XR)^{-1}(XR)^{T}y$</font>  $\omega_{new} = XR(X^T X)^{-1} X^T y$
<br>Откроем скобки: <font color='lightgray'>$\omega_{new}=((X^{T}R^{T}XR)^{-1}X^{T}R^{T}y$</font>    $\omega_{new} = (R^T (X^T X) R)^{-1} (XR)^T y$
<br>*Далее:* $\omega_{new} = (R^T)^{-1} (X^T X)^{-1} R^{-1} X^{T}R^T y$
<br>Выражение $R^{T}(R^{T})^{-1}$ можно сократить, т к это формула единичной матрицы,а матрица умножаясь на единичную равна себе. <br>Получаем: $\omega_{new}=R^{-1}(X^{T}X)^{-1}X^{T}y $
<br> Таким образом, выражение можно привести к виду: $\omega_{new}=R^{-1}\omega$


 **Вопрос:** изменится ли качество линейной регрессии? 

**Ответ:** не изменится.

**Обоснование:** 
Качество модели от умножения  на обратимую матрицу не изменится по следующей причине:
<br>Произведение матрицы состоит из всех возможных комбинаций скалярных произведений вектор-строк матрицы Features и рандомно сгенерированной матрицы. Таким образом мы произведем умножение каждого набора признаков (строка-вектор) из Features на одинаковые наборы векторов (столбец - вектор) из рандомной матрицы и запишем их в новый вектор - строку. Соответственно итоговые веса каждого такого вектора будут примерно соотносится с весами набора признаков для каждого нового набора векторов в результирующей матрице.


<font color="lightgray">**Для квадратных матриц:**
Предсказания высчитываются по формуле: $a=X_{1}\omega$
<br>Подставляем в выражение формулу обучения: $w = (X^T X)^{-1} X^T y$
<br>Получаем: $a= X_{1}(X^{T}X)^{-1}X^{T}y$, открываем скобки:$ a = X_{1}X^{-1}(X^{T})^{-1}X^{T}y$, сокращаем $(X^{T})^{-1}X^{T}$, это формула единичной матрицы,а матрица умножаясь на единичную равна себе.
<br>В итоге получаем: $a = X_{1}X^{-1}y$
<br> Далее введем в формулу рандомную матрицу, обозначенную как **R**, на которую мы умножили исходную матрицу признаков. Раскроем скобки и сократим выражение. 
$a_{new} = (X_{1}R)((XR)^{T}XR)^{-1}(XR)^{T}y$ 
1) $(X_{1}R)(XR)^{-1}((XR)^{T})^{-1}(XR)^{T}y$

2) $(X_{1}R)R^{-1}X^{-1}(X^{T})^{-1}(R^{T})^{-1}R^{T}X^{T}y$ 

3) $X_{1}RR^{-1}X^{-1}(X^{T})^{-1}(R^{T})^{-1}R^{T}X^{T}y =  X_{1}X^{-1}y$

<br>**В итоге получаем:** предсказания с преобразованной матрицей $a_{new} = X_{1}X^{-1}y$, равны предсказаниям с исходной матрицей **а**.</font>

**Исправление:**

$$
a = Xw = XEw = XPP^{-1}w = (XP)P^{-1}w = (XP)w'
$$

$$
w' = ((XP)^T XP)^{-1} (XP)^T y
$$
$$
w' = (P^T (X^T X) P)^{-1} (XP)^T y
$$
$$
w' = (P^T (X^T X) P)^{-1} P^T X^T y
$$
$$
w' = P^{-1} (X^T X)^{-1} (P^T)^{-1} P^T X^T y
$$
$$
....
$$

$a=XRw$

$w = ((XR)^TXR)^{-1}(XR)^Ty$

Подставим w в a:

$a = XR(((XR)^TXR)^{-1}(XR)^Ty)$

Раскроем транспортирование по свойству: $(XR)^T = R^TX^T$
$a = XR(((R^TX^TXR)^{-1}R^TX^Ty)$

Используем свойство $(XR)^{-1} = R^{-1}X^-1$:

$a = XR(((X^TXR)^{-1}(R^T)^{-1}R^TX^Ty)$
$a = XRR^{-1}(((X^TX)^{-1}(R^T)^{-1}R^TX^Ty)$

$a = XE(X^TX)^{-1}EX^Ty$

Сокращаем единичные матрицы:
$a = X(X^TX)^{-1}X^Ty$
Выделяем формулу w  приходим к исходному виду a:
$a = X(X^T)^{-1}X^Ty = Xw$

***Вывод:***
- Качество линейной регрессии не изменилось, т к предсказания с преобразованной матрицей равны предсказаниям с исходной матрицей и R-метрики примерно равны;
- связь параметров линейной регрессии в исходной задаче и в преобразованной можно выразить в формуле: $$\omega_{new}=R^{-1}\omega$$

## Алгоритм преобразования

Используем метод *encode/decode*.
<br>***Сформируем функции с алгоритмами кодирования и декодирования.***


In [17]:
def encode(features):
    random_matrix = np.random.normal(0, 1, (features.shape[1], features.shape[1]))
    encoded_features = features.dot(random_matrix)
    return encoded_features

def decode(encoded_features):
    decoded_features = round(abs(encoded_features.dot(np.linalg.inv(random_matrix))))
    for i in decoded_features.iloc[:, [0, 3]]:
        decoded_features[i] = decoded_features[i].astype(int)
    decoded_features.columns = features.columns
    return decoded_features


Применим функцию кодирования к матрице features:

In [18]:
encode(features)

,0,1,2,3
0,11568.312111,-16861.734183,20719.570391,-52144.198917
1,8868.501056,-12909.406369,15868.485729,-39981.262298
2,4902.382788,-7131.246704,8768.932959,-22102.761280
3,9720.438905,-14184.180441,17423.301681,-43806.466152
4,6089.916790,-8869.107259,10900.926047,-27454.500534
...,...,...,...,...
4995,8325.797789,-12138.227465,14912.122464,-37526.888220
4996,12217.536308,-17817.038825,21893.504024,-55063.395968
4997,7903.414749,-11530.027478,14162.581081,-35619.560466
4998,7624.938354,-11123.707163,13657.926188,-34368.469870


Применим функцию дешифрования к зашифрованной матрице *encoded_features* и возвращающая исходную матрицу *features*:

In [19]:
decode(encoded_features)

,Пол,Возраст,Зарплата,Члены семьи
0,1,41.0,49600.0,1
1,0,46.0,38000.0,1
2,0,29.0,21000.0,0
3,0,21.0,41700.0,2
4,1,28.0,26100.0,0
...,...,...,...,...
4995,0,28.0,35700.0,2
4996,0,34.0,52400.0,1
4997,0,20.0,33900.0,2
4998,1,22.0,32700.0,3


Сравним с исходным features:

In [20]:
features

,Пол,Возраст,Зарплата,Члены семьи
0,1,41,49600.0,1
1,0,46,38000.0,1
2,0,29,21000.0,0
3,0,21,41700.0,2
4,1,28,26100.0,0
...,...,...,...,...
4995,0,28,35700.0,2
4996,0,34,52400.0,1
4997,0,20,33900.0,2
4998,1,22,32700.0,3


**Вывод:** Кодирование и декодирование прошло удачно, без изменения данных.

## Проверка алгоритма

Проверим, что качество линейной регрессии из sklearn не отличается до и после преобразования. Применим метрику R2.

Напишем модель линейной регрессии sklearn и обучим на 2 матрицах:
- исходной features
- кодированной encoded_features

In [21]:
sklearn_model = LinearRegression().fit(features, target)
pred = sklearn_model.predict(features)
r2_skl_features = r2_score(target, pred)
r2_skl_features

0.4249455028666801

In [22]:
sklearn_model.fit(encoded_features, target)
pred = sklearn_model.predict(encoded_features)
r2_skl_encoded = r2_score(target, pred)
r2_skl_encoded


0.4249455028666843

In [23]:
if round(r2_skl_encoded, 3) == round(r2_skl_features, 3):
    print('R-метрики до и после кодирования примерно равны')
else:
    print('R-метрики до и после кодирования не равны')
    


R-метрики до и после кодирования примерно равны


Сравним R-метрики по двум выборкам линейной регрессии *sklearn* и вычисленные с помощью функций из класса *my_L_regression*:

In [24]:
if round(r2_skl_encoded, 3) == round(r2_skl_features, 3) == round(r2_first, 3) == round(r2_second, 3):
    print('R-метрики примерно равны')
else:
    print('R-метрики не равны')
    
    

R-метрики примерно равны


**Вывод:** Качество модели  линейной регрессии sklearn по двум наборам признаков - до и после преобразования исходных признаков - совпадает. Так же, R-метрики по двум выборкам линейной регрессии *sklearn* и вычисленные с помощью функций из класса *my_L_regression* примерно равны, что говорит о ее равнозначности  линейной регрессии из sklearn.

## Общий вывод

В прощессе выполнения работы был изучен файл с исходными данными: выполнена проверка на наличие пропусков, дубликатов, адекватности данных. Выполнен перевод данных в целочисленный тип в соотвтетсвии с содержанием столбцов. Аномалий и пропусков не обнаружено. 
Далее были определены целевые признаки, сформированы выборки *features* и *target* выборки из данных и проверим работу линейной регрессии из scikit-learn, сравнили ее результат с результатом вычисления линенйной регрессии по формуле. Так же, в процессе работы было доказано путем теоретических вычислений и практических манипуляций(формирование класса * my_L_regression* на основе формулы и обучение исходной матрицы и матрицы, умноженной на случайную с помощью заключенных в нем функций), что качество модели от умножения на обратимую матрицу не изменится, т е  метрика R2 не ухудшится. Далее, путем преобразования формул согласно проведенным вычислниям была показана связь  параметров линейной регрессии в исходной задаче и в преобразованной. Был сформирован алгоритм преобразования с помощью метода *encode/decode* - были сформированы функции кодирования и декодирования, подходящие для использования с целью зашифровки и восстановления исходных данных. Выполнено обучение моделей линейной регрессии из sklearn на двух матрицах - исходной features и кодированной encoded_features. Вычислены R - метрики, которые при сравнении между собой и с метриками, вычисленными на основе полученных с помощью класса *my_L_regression* данных. Таки образом, доказана равнозначность  линейной регрессии из sklearn и функций классса *my_L_regression*.

**Финальные выводы:**
- при умножении признаков на обратимую матрицу, качество линейной регрессии не меняется; 
- использованный в работе алгоритм преобразования данных подходит шифрования признаков;
- полученные путем шифрования данные так же подходят для предсказаний с помощью линейной регресии;
- алгоритм декодирования данных справляется со своей задачей;
- т к умножение идет на рандомную матрицу, при каждом использовании алгоритма будет сформирован новый уникальный "ключ", что затрудняет расшифровку данных.